In [6]:
import rospy
rospy.init_node('blockly_server', anonymous=True)
# Copied imports for setup_miro here to allow execution of other functions
from miro_msgs.msg import platform_control
from miro_constants import miro
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Range, Image
import sys
import rospy
import subprocess
import rosnode
import numpy as np
import cv2
import time
import os
import math
import rospkg
global q, pub, current_robot, rate
# These global variables need to be added inside of each function

In [2]:
# Colour: 230
def setup_miro():
    global q, pub, current_robot, rate
    from miro_constants import miro
    from miro_msgs.msg import platform_control
    from geometry_msgs.msg import Twist
    import time
    import sys
    import rospy
    import subprocess
    import rosnode
    import math
    import numpy as np
    import cv2
    import os
    import rospkg

    if ('current_robot' not in locals()):
        current_robot = 'sim01'

    pub = rospy.Publisher('/miro/' + current_robot + '/platform/control',
        platform_control, queue_size=10)
    rate = rospy.Rate(10)
    q = platform_control()

In [3]:
setup_miro()

In [ ]:
# Colour: 230
def move_backward():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.x = 0
    body_vel.linear.x = -200
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)	#Allow time for the move to be executed

In [ ]:
move_backward()

In [ ]:
# Colour: 230
def move_forward():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.x = 0
    body_vel.linear.x = +200
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)	#Allow time for the move to be executed

In [ ]:
move_forward()

In [ ]:
# Colour: 230
def turn_left():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.z = +0.785398
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)	#Allow time for the move to be executed

In [ ]:
turn_left()

In [ ]:
# Colour: 230
def turn_right():
    global q, pub, current_robot, rate
    body_vel = Twist()
    body_vel.angular.z = -0.785398
    q.body_vel = body_vel

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)	#Allow time for the move to be executed

In [ ]:
turn_right()

In [ ]:
# Colour: 160
def get_distance():
    global q, pub, current_robot, rate
    msg_distance = rospy.wait_for_message('/miro/' + current_robot + '/sensors/sonar_range', Range, timeout=1)
    return msg_distance.range

In [ ]:
get_distance()

In [ ]:
# Interface: dropdown
# UserParameters: [Up, Down, Centre]
# CodeParameters: [L_UP, L_DOWN, L_CENTRE]
# Colour: 180
def lift_neck(dropdown_lift):
    global q, pub, current_robot, rate
    if dropdown_lift == 'L_UP':
        direction = miro.MIRO_LIFT_MIN_RAD
    elif dropdown_lift == 'L_DOWN':
        direction = miro.MIRO_LIFT_MAX_RAD
    else:
        direction = (miro.MIRO_LIFT_MIN_RAD + miro.MIRO_LIFT_MAX_RAD )/2
    q.body_vel = Twist()
    q.body_config[1] = direction
    q.body_config_speed[1] = miro.MIRO_P2U_W_LEAN_SPEED_INF

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)#Allow time for the move to be executed

In [ ]:
lift_neck("L_DOWN")
lift_neck("L_UP")
lift_neck("L_CENTRE")

In [ ]:
# Interface: dropdown
# UserParameters: [Up, Down, Centre]
# CodeParameters: [P_UP, P_DOWN, P_CENTRE]
# Colour: 180
def pitch_neck(dropdown_pitch):
    global q, pub, current_robot, rate
    if dropdown_pitch == 'P_UP':
        direction = miro.MIRO_PITCH_MIN_RAD
    elif dropdown_pitch == 'P_DOWN':
        direction = miro.MIRO_PITCH_MAX_RAD
    else:
        direction = (miro.MIRO_PITCH_MIN_RAD + miro.MIRO_PITCH_MAX_RAD )/2
    
    q.body_vel = Twist()
    q.body_config[3] = direction
    q.body_config_speed[3] = miro.MIRO_P2U_W_LEAN_SPEED_INF

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)	#Allow time for the move to be executed

In [ ]:
pitch_neck("P_DOWN")
pitch_neck("P_UP")
pitch_neck("P_CENTRE")

In [ ]:
# Interface: dropdown
# UserParameters: [Right, Left, Centre]
# CodeParameters: [Y_RIGHT, Y_LEFT, Y_CENTRE]
# Colour: 180
def yaw_neck(dropdown_yaw):
    global q, pub, current_robot, rate
    if dropdown_yaw == 'Y_RIGHT':
        direction = miro.MIRO_YAW_MAX_RAD
    elif dropdown_yaw == 'Y_LEFT':
        direction = miro.MIRO_YAW_MIN_RAD
    else:
        direction = (miro.MIRO_YAW_MIN_RAD + miro.MIRO_YAW_MAX_RAD )/2
    
    q.body_vel = Twist()
    q.body_config[2] = direction
    q.body_config_speed[2] = miro.MIRO_P2U_W_LEAN_SPEED_INF

    #ensures that at least one node is connected before sending message
    while(pub.get_num_connections() == 0):
        rate.sleep()
    pub.publish(q)
    time.sleep(3)	#Allow time for the move to be executed

In [ ]:
yaw_neck("Y_RIGHT")
yaw_neck("Y_LEFT")
yaw_neck("Y_CENTRE")

In [ ]:
# Colour: 180
# Show Image
def capture_image():
    timestr = time.strftime("%d-%m-%Y_%H-%M-%S.png")

    # Ros Messages	 
    from sensor_msgs.msg import CompressedImage

    msg_image = rospy.wait_for_message('/miro/'  + current_robot + '/platform/caml/compressed', CompressedImage, timeout=7)
    np_arr = np.frombuffer(msg_image.data, np.uint8)
    image_np = cv2.imdecode(np_arr, 1) #cv2.CV_LOAD_IMAGE_COLOR


    rospack = rospkg.RosPack()
    images_path = '/usr/local/src/robot/blockly_ws/install_isolated/share/robot_blockly/frontend/pages/images/'
    cv2.imwrite(images_path+ 'image_' + timestr, image_np)

    #cv2.imwrite('/home/erle/spider_ws/install_isolated/share/robot_blockly/frontend/pages/images/image_' + timestr, image_np)
    #images_path = "/home/erle/spider_ws/install_isolated/share/robot_blockly/frontend/pages/images/"

    files = len(os.listdir(images_path)) #amount of files in /frontend/images/ folder

    if files > 7 : #allow 5 images max
        os.system("find "+images_path+" -name '*.png' | xargs ls -t | tail -n 1 | xargs rm")#remove oldest image


In [ ]:
capture_image()

In [39]:
# Colour: 180
def get_cam_colour():
    colorBGR = "0,100,0"
    global q, pub, current_robot, rate
    #colorBGR (colour picker input)
    inB = int(colorBGR.split(',')[0])
    inG = int(colorBGR.split(',')[1])
    inR = int(colorBGR.split(',')[2])

    image_msg = rospy.wait_for_message('/miro/' + current_robot + '/platform/caml', Image, timeout=10)

    h  = image_msg.height
    w  = image_msg.width
    data = np.frombuffer(image_msg.data, np.uint8)

    # Find central(-ish) pixel, bottom right one in a even x even image
    position = (h/2) * w + w/2

    # get the 3 colour components of the central pixel
    B = data[position]
    G = data[position + 1]
    R = data[position + 2]

    th = 15
    result = False
    if abs(inB - B) <= th and abs(inG - G) <= th and abs(inR - R) <= th:
        result = True

In [40]:
get_cam_colour("0,100,0")

False

In [ ]:
setup_miro()
move_forwards()
turn_left()
move_backwards()
turn_right()
lift_neck()
yaw_neck()
pitch_neck()
